In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [3]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type

        with open('./data/chat_feature_real.pickle',"rb") as f1:  
            self.chat_result=pickle.load(f1)
        
        with open('./LSTM_feature_extractor_Audio_13_ver2/features',"rb") as f2:  
            self.audio=json.load(f2)
        with open('./video_raw_feature_sum_moving_average/video_feature.pickle',"rb") as f2:  
            self.video=pickle.load(f2)
        with open('./label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.real_result[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(23): #7 : window size
                s_window=[]
                if vframe+idx<len(self.real_result[game_id]):
#                     s_window+=((self.chat_result[game_id][vframe+idx]))#vframe의 chat
#                     s_window+=(self.audio[game_id][vframe+idx])#vframe의 image
                    s_window+=self.video[game_id][vframe+idx]
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*128
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [4]:
train=Mul_data('train')
val=Mul_data('val')

print(train[100])
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844412722519367', array([[-0.08037048, -0.02835998,  0.14703284, ..., -0.04624566,
        -0.14462242, -0.06322261],
       [-0.0794142 , -0.02593253,  0.14272448, ..., -0.04575907,
        -0.1425308 , -0.06174482],
       [-0.07915168, -0.02541708,  0.14164425, ..., -0.04569324,
        -0.14201146, -0.06145829],
       ...,
       [-0.0791397 , -0.02482486,  0.14111769, ..., -0.04539137,
        -0.1416546 , -0.0609643 ],
       [-0.07862069, -0.02350517,  0.13873826, ..., -0.04509101,
        -0.14048611, -0.06013102],
       [-0.07838232, -0.02296594,  0.13767907, ..., -0.04498608,
        -0.13997285, -0.0597976 ]]), 0)


In [6]:
train[100][1].shape

(23, 384)

In [5]:
input_size=128
hidden_size=128
length=7
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        feature = out[:,-1,:]
        out = self._lin(out[:,-1,:])
        return out,feature

In [6]:
model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [8]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [9]:
weight_dir='./V_mov/'


In [10]:
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(100):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out,_=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out,_=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            







0
epoch 0 train_loss : 0 , val_loss : 0.3739182949066162,p 0.5896955503512881, r 0.8337748344370861, f 0.690809327846365

1
epoch 1 train_loss : 0 , val_loss : 0.39305758476257324,p 0.5221510883482714, r 0.9002207505518763, f 0.6609400324149108

2
epoch 2 train_loss : 0 , val_loss : 0.3341986835002899,p 0.5667283862697622, r 0.8783664459161148, f 0.6889446801142759

3


KeyboardInterrupt: 

In [23]:
test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output,_=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 0 27 0 tensor(5) tensor(5)
8 0 23 1 tensor(8) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 1 22 0 tensor(10) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
2 0 16 14 tensor(2) tensor(16)
14 0 15 3 tensor(14) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 6 14 3 tensor(15) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 9 23 0 tensor(9) tensor(0)
0 2 30 0 tensor(2) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 2 23 0 tensor(9) tensor(7)
14 0 14 4 tensor(14) tensor(18)
6 1 23 2 tensor(7) tensor(8)
0 13 19 0 tensor(13) tensor(0)
10 0 19 3 tensor

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 9 23 0 tensor(9) tensor(0)
17 5 10 0 tensor(22) tensor(17)
8 5 14 5 tensor(13) tensor(13)
10 5 17 0 tensor(15) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 25 7 0 tensor(25) tensor(0)
0 13 19 0 tensor(13) tensor(0)
14 9 9 0 tensor(23) tensor(14)
4 0 21 7 tensor(4) tensor(11)
9 19 3 1 tensor(28) tensor(10)
0 0 30 2 tensor(0) tensor(2)
0 3 29 0 tensor(3) tensor(0)
0 13 19 0 tensor(13) tensor(0)
0 0 32 0 tensor(0) tensor(0)
15 8 9 0 tensor(23) tensor(15)
7 0 23 2 tensor(7) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 2 30 0 tensor(2) tensor(0)
0 10 22 0 tensor(10) tensor(0)
3 18 11 0 tensor(21) tensor(3)
31 0 0 1 tensor(31) tensor(32)
18 0 12 2 tensor(18) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
28 0 3 1 tensor(28) tensor(29)
8 0 20 4 tensor(8) tensor(12)
0 12 20 0 tensor(12) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 13 19 0

0 0 32 0 tensor(0) tensor(0)
0 14 18 0 tensor(14) tensor(0)
10 3 18 1 tensor(13) tensor(11)
0 11 21 0 tensor(11) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 0 23 3 tensor(6) tensor(9)
7 0 23 2 tensor(7) tensor(9)
0 3 29 0 tensor(3) tensor(0)
4 6 22 0 tensor(10) tensor(4)
0 27 5 0 tensor(27) tensor(0)
8 15 7 2 tensor(23) tensor(10)
11 5 15 1 tensor(16) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
3 5 24 0 tensor(8) tensor(3)
20 5 4 3 tensor(25) tensor(23)
8 8 15 1 tensor(16) tensor(9)
0 4 28 0 tensor(4) tensor(0)
0 2 30 0 tensor(2) tensor(0)
0 10 15 7 tensor(10) tensor(7)
0 0 26 6 tensor(0) tensor(6)
12 9 11 0 tensor(21) tensor(12)
11 0 12 9 tensor(11) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 1 22 0 tensor(10) tensor(9)
7 13 12 0 tensor(20) tensor(7)
0 1 29 2 tensor(1) tensor(2)
0 0 10 22 tensor(0) tensor(22)
26 0 6 0 tensor(26) tensor(26)
8 2 22 0 te

0 6 26 0 tensor(6) tensor(0)
28 4 0 0 tensor(32) tensor(28)
3 13 14 2 tensor(16) tensor(5)
0 2 30 0 tensor(2) tensor(0)
23 3 6 0 tensor(26) tensor(23)
13 3 15 1 tensor(16) tensor(14)
6 3 23 0 tensor(9) tensor(6)
7 7 18 0 tensor(14) tensor(7)
29 1 2 0 tensor(30) tensor(29)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 6 26 0 tensor(6) tensor(0)
22 8 2 0 tensor(30) tensor(22)
7 7 18 0 tensor(14) tensor(7)
0 0 32 0 tensor(0) tensor(0)
6 4 22 0 tensor(10) tensor(6)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 0 20 1 tensor(11) tensor(12)
19 7 4 2 tensor(26) tensor(21)
8 7 17 0 tensor(15) tensor(8)
0 0 32 0 tensor(0) tensor(0)
16 0 16 0 tensor(16) tensor(16)
26 0 6 0 tensor(26) tensor(26)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
3 0 10 19 tensor(3) tensor(22)
22 0 3 7 tensor(22) tensor(29)
0 0 32 0 ten

In [24]:
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
with open('./label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

102844212431058132
precision : 0.6316831683168317, recall : 0.8372703412073491, f1 : 0.7200902934537246, accuracy : 0.8993506493506493
102844341902586509
precision : 0.5501165501165501, recall : 0.932806324110672, f1 : 0.6920821114369502, accuracy : 0.90177736202058
102844401152267937
precision : 0.6601073345259392, recall : 0.917910447761194, f1 : 0.7679500520291362, accuracy : 0.9025775447793797
102844212430927059
precision : 0.6707466340269278, recall : 0.8303030303030303, f1 : 0.7420446851726472, accuracy : 0.899815934788325
102844412708953395
precision : 0.5943152454780362, recall : 0.8333333333333334, f1 : 0.693815987933635, accuracy : 0.9012645914396887
102844212429944013
precision : 0.6227180527383367, recall : 0.8274932614555256, f1 : 0.710648148148148, accuracy : 0.8778103616813294
102844341912679064
precision : 0.5638629283489096, recall : 0.8008849557522124, f1 : 0.6617915904936015, accuracy : 0.904639175257732
102844235753749959
precision : 0.5794573643410853, recall : 0.7

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [11]:
x=[]

In [9]:
with open('../data/chat_feature_pred_128_train.json',"rb") as f1:  
    chat_result=json.load(f1)

In [24]:
len(chat_result['102844235753356742'])

1654

In [21]:
        with open('../data/audio_energy_2_normaized.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [25]:
audio_result['102844235753356742'][1]

4.277879204882054e-07

In [8]:
        with open('../data/audio_H.pickle',"rb") as f2:  
            image_result=pickle.load(f2)

In [10]:
len(image_result['102844235753356742'])

16571